In [41]:
import boto3
import spotipy
import json
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials

In [ ]:
client = boto3.client('lambda')
response = client.invoke(FunctionName='spotifyLambda', InvocationType='RequestResponse')
data = json.loads(response['Payload'].read())
token = json.loads(data['body'])['token']

In [ ]:
sp = spotipy.Spotify(token)

In [45]:
targetPlaylists = ['1uI3NIn044CEKD9TWu1Ue8', '1JAbly1IIWtRg7nJ6r3Jc8', # Ken Ishii, Jennifer Cardinii
                   '3b7yq4QSnOTkUd0y0mRHWY', '1sPcU9xKa4je10L25yiuP4', # Kamma & Masalo, Marcel Dettman
                   '4vV44l7tTjWO91qfWhtRnv', '1S7L8yPQdg8PJxcTL3PBEJ', # Pan-Pot, Sherelle
                    '6lhwu8MQQ5rwRmZpigkXQy', '1v2QmoXC9RM8sT6iS477Hf', # Ellen Allien, Shy FX
                    '1LKGVHraQ6kAakMS06oqaj', '2Znwqk4MvlwPAqbD7V6afm', # Kelly Lee Owens, Gerd Janson
                    '53GktLN64FUR1Y1GLRmWgl', '3zbTCeuAs5SBLBU41DvifR', # Nina Kraviz, Todd Terry
                    '4bmYqCOdbuTdmDPxD2uNKZ', '5z8ovj124l7kxcgwdKfvce', # Dixon, Qrion
                    '6vdUshyM4sRJa4M7UqLbLo', '1s23VsGfOOb1vMusO1xGNO', # Jayda G, Amelie Lens
                    '6FvxOLHUYlf8y5On5Fs6FV', '64Vp25NhvFNSJertFIpS1N', # Aluna, Green Velvet
                    '6q4npevstoEIXA2j68CNdE', '2ftOkVeVtKZJYUauGETLL0', # Honey Dijon, Green Velvet
                    '0fzwhHQXDxvrEo73nx70a5', '2CDQ5oydb2lNh1lUjuWNCP', # MK, Carl Cox
                    '1ofsXslAe9rBBNNVfOsKPX' # Black Coffee
                   ]

In [14]:
def artistParse(artists):
    artist = []
    for a in artists:
        artist.append(a['name'])
    return ', '.join(artist)

In [56]:
data = []
for playlistID in targetPlaylists:
    playlist = sp.playlist(playlistID)
    playlistTracks = playlist['tracks']['items']
    DJ = playlist['name']
    
    for track in playlistTracks:
        artist = artistParse(track['track']['artists'])
        trackname = track['track']['name']
        album = track['track']['album']['name']
        album_uid = track['track']['album']['id']
        duration_ms = track['track']['duration_ms']
        duration_seconds = round(duration_ms/1000.0)
        uid = track['track']['id']
    
        data.append({
            'spot_artists': artist,
            'album': album,
            'spot_name': trackname,
            'spot_duration': duration_seconds,
            'album_uid': album_uid,
            'track_uid': uid,
            'DJ': DJ
    #       'yt_id': yt_id,
    #       'yt_title': yt_title,
    #       'yt_duration': yt_duration
        })

In [61]:
df = pd.DataFrame(data=data, columns=data[0].keys())

In [62]:
df

,spot_artists,album,spot_name,spot_duration,album_uid,track_uid,DJ
0,Vinicius Honorio,Wasteland EP,Wasteland,390,06NY9fLUaJEwB0bTDKcAYy,3yWq1MBgt3iLALZSHYczV9,Ken Ishii
1,Chicago Loop,Push The Feeling EP,Clap Your Hands,397,6eHMNuuUf2SexfsS8arhNr,7jde87b79aUrNJ6OuGkfDO,Ken Ishii
2,"Quadra 163, Osunlade",Spin Coaster EP (Including Osunlade Rework),Spin Coaster - Osunlade Rework,551,0SsjzUnzg6V5g59SfqzvlD,4pe9kcedTykmTDHTLkTwVb,Ken Ishii
3,Antony Dupont,"The Sound of Metrohm, Vol. 1",Take Me Away,409,6xF9SuPHJpAWUO1CQqvueK,6lbFejCWExGPjQ78MbTJGF,Ken Ishii
4,"Alberto Ruiz, Xenia (UA)",Off,Off - Original Stick,330,6Wla6vLzjdZjHXocigyi9K,2112eAdXYAcMbd61BizFM6,Ken Ishii
...,...,...,...,...,...,...,...
1344,"Re.You, Oluhle, Aaaron, Floyd Lavine",Maison 'The Remixes',Inyani - Floyd Lavine,445,0sdpLNuVlwTakX1YPljA1P,2gezPlMhekV6viEVRFSeYz,Black Coffee
1345,"G. Washington, Tabia",Uyisilo,Uyisilo,371,4xf4KGY3jRi13TTzZAgaKv,2NuUGkpdUfiXwI6WqqLwkm,Black Coffee
1346,"AWEN, Caiiro, Enoo Napa",Your Voice,Your Voice - Enoo Napa Remix,450,13czSpTI2okjVxVQxnTpK2,2kCZ8eOCU6nMnXp3FzyhUE,Black Coffee
1347,"Web Web, Michael Cleis",25 Compost Records,Maroc Blues - Michel Cleis Remix,546,4IajSl8vaCxyrgxTBPdD6i,15Lb2SPiJhJADZJq5AewkR,Black Coffee


In [65]:
df[df.duplicated(subset=['track_uid']) == True]

,spot_artists,album,spot_name,spot_duration,album_uid,track_uid,DJ
231,Anthony Rother,Stellarator Hyperway,Stellarator,430,4psxm91qSSftppYxClfS0F,3qlMVTsnzChWTX2Lhv4oRI,Marcel Dettmann
403,"Boy Harsher, Marcel Dettmann",Come Closer (Marcel Dettmann Remix),Come Closer - Marcel Dettmann Remix,400,5qNMeOa5MKx1y6GMXTbYgY,55sQYh7H8G78OygiCASEKy,Ellen Allien
405,Kendal,Inferno,Intacto,376,4tvnchNiDvrGs31kJLjFfs,6oO4m6VjpWTRvTtMmhNxdt,Ellen Allien
460,"Master Plan, Enzo Elia",Electric Baile / Pushin' Too Hard (Gerd Janson...,Electric Baile - Enzo Elia Vocal Edit,322,1xyEt7Vp57IFy1xrsyI2eV,1GCzhYzCo1FI08ixz1zVSn,Gerd Janson
464,Feater,Money,Money,321,0M5zCOCnMxM3gFNReS5VrW,4NGdIkvsJ6kVbQS6yW419J,Gerd Janson
466,"Sirens Of Lesbos, Gerd Janson",Zeus Remixes,Zeus - Gerd Janson Symposion Remix,394,09MEGrSlSOpO486rRvR247,0fEu7sn6rssTJVnMmCmmt8,Gerd Janson
467,Johannes Volk,Extra Dimensions,Extra Dimensions,326,35beRwYqwrhtq7NjXn4PHr,1AN6iETDM1zDH4gNGDBuT0,Gerd Janson
475,Johannes Albert,Yoga At The Dream Haus EP,More Action Pleeease!,222,13YnwgU6fXwDJmdiiFZCpo,6bD5vnX4Gf3ayEFr3xEUfo,Gerd Janson
477,"Reznik, good guy mikesh",Human Factor,Human Factor,375,4CbqdW8Ih37rvpqcTIZwA1,5AGAinKC7baBY4OWUsyRpK,Gerd Janson
491,Vril,Bad Manners 4,Scalar,503,7KXPfxFFeyzYLqxPqvUtlE,1mB25QcfqfEtTPD3B8O67Y,Gerd Janson


In [66]:
df[df['track_uid'] == '7wAVCSMQa3WyFFhfGfPDNP']

,spot_artists,album,spot_name,spot_duration,album_uid,track_uid,DJ
729,"Howling, Rampa, RY X, Frank Wiedemann",Bind (Remixes),Bind - Rampa Remix,379,0ajoDZRY4n3a7KIq9F7qC3,7wAVCSMQa3WyFFhfGfPDNP,Qrion
1311,"Howling, Rampa, RY X, Frank Wiedemann",Bind (Remixes),Bind - Rampa Remix,379,0ajoDZRY4n3a7KIq9F7qC3,7wAVCSMQa3WyFFhfGfPDNP,Black Coffee
